###**Q1**.  Use Lucidchart to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [ ]:
# 2.2 Create the database and save it in a file called store.db, and create the four tables as described above.
# Your code here . . .
import sqlite3
conn = sqlite3.connect('store.db')
c = conn.cursor()

c.execute('''DROP TABLE IF EXISTS products''')
c.execute('''DROP TABLE IF EXISTS customers''')
c.execute('''DROP TABLE IF EXISTS purchase_order''')
c.execute('''DROP TABLE IF EXISTS order_item''')

SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS products (
             product_id INTEGER PRIMARY KEY,
             name TEXT NOT NULL,
             price REAL
             )'''

c.execute(SQL_CreateTable)
SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS customers (
             customer_id INTEGER PRIMARY KEY,
             name TEXT NOT NULL,
             email TEXT NOT NULL
             )'''

c.execute(SQL_CreateTable)
SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS purchase_order (
             order_id INTEGER PRIMARY KEY,
             customer_id INTEGER,
             date TEXT NOT NULL,
             FOREIGN KEY (customer_ID)
                REFERENCES customer(customer_ID)
                    ON DELETE CASCADE
             )'''

c.execute(SQL_CreateTable)
SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS order_item(
             order_id INTEGER,
             product_id INTEGER,
             quantity INTEGER,
             FOREIGN KEY (order_id)
                REFERENCES purchase_order(order_id)
                    ON DELETE CASCADE,
             FOREIGN KEY (product_id)
                REFERENCES product(product_id)
                    ON DELETE CASCADE
             )'''

c.execute(SQL_CreateTable)

In [ ]:
# 2.3 Load the data in the csv files into the corresponding table.
# Your code here . . .
import pandas as pd
products = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/product.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv')
purchase_order = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv')
order_item = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv')

In [ ]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
# Your code here . . .
products.to_sql('products', conn, if_exists='append', index=False)
customers.to_sql('customers', conn, if_exists='append', index=False)
purchase_order.to_sql('purchase_order', conn, if_exists='append', index=False)
order_item.to_sql('order_item', conn, if_exists='append', index=False)


4

In [ ]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
# Your code here . . .
c.execute("SELECT * FROM products")
results = c.fetchall()
print(results)

c.execute("SELECT * FROM customers")
results = c.fetchall()
print(results)

c.execute("SELECT * FROM purchase_order")
results = c.fetchall()
print(results)

c.execute("SELECT * FROM order_item")
results = c.fetchall()
print(results)

[(0, 'bicycle', 400.0), (1, 'helmet', 45.0), (2, 'gloves', 23.0), (3, 'chain', 48.0)]
[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu')]
[(0, 0, '2020-10-19'), (1, 0, '2020-10-20'), (2, 1, '2020-10-20')]
[(0, 0, 1), (0, 1, 1), (1, 2, 2), (1, 3, 1)]


In [ ]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
# Your code here . . .
query = '''
          SELECT p.date, pr.name, o.quantity, c.name
          FROM purchase_order p
          JOIN order_item o ON p.order_id = o.order_id
          JOIN products pr ON o.product_id = pr.product_id
          JOIN customers c ON p.customer_id = c.customer_id
          '''

#c.execute(query)
#results = c.fetchall()
#print(results)
pd.read_sql_query(query, conn)

,date,name,quantity,name
0,2020-10-19,bicycle,1,Wendy Lee
1,2020-10-19,helmet,1,Wendy Lee
2,2020-10-20,gloves,2,Wendy Lee
3,2020-10-20,chain,1,Wendy Lee


In [ ]:
# Additional tasks
# Appending a new row to one of the tables
c.execute('''
          INSERT INTO products (product_id, name, price)
          VALUES (4, 'New Product', 10.99)
          ''')

# Deleting row(s) with "XXX"
c.execute('''
          DELETE FROM products
          WHERE name = 'chain'
          ''')

# Updating row(s) with "XXX"

pd.read_sql_query('SELECT * FROM products', conn)

,product_id,name,price
0,0,bicycle,400.00
1,1,helmet,45.00
2,2,gloves,23.00
3,4,New Product,10.99
